In [1]:
import numpy as np
import zipfile
import matplotlib.pyplot as plt
from PIL import Image
import os
import tensorflow as tf

x_train, y_train = np.zeros([2688, 224, 224,3]), np.zeros([2688])
x_test, y_test = np.zeros([896, 224, 224,3]), np.zeros([896])

folder = "IITD_database/people"
i = 0
for people in range(1,225):
    person_folder = folder + "/" + "0"*(3-len(str(people))) + str(people)
    file_names = os.listdir(person_folder)
    for num_image in range(1,7):
        path = person_folder + "/" + file_names[num_image-1]
        image = Image.open(path)
        image = image.resize((224,224))
        image = np.asarray(image)
        #x = image.reshape((1,224,224,3))
        row_num = i*6 + num_image -1
        x_train[row_num,:,:,:] = image
        y_train[row_num] = people
        
        flip = np.flip(x_train[i],1)
        flip = np.resize(flip, (1,224,224,3))
        x_train[row_num+1344,:,:,:] = flip
        y_train[row_num+1344] = people
    i += 1
    
i = 0
for people in range(1,225):
    person_folder = folder + "/" + "0"*(3-len(str(people))) + str(people)
    file_names = os.listdir(person_folder)
    for num_image in range(7,11):
        path = person_folder + "/" + file_names[num_image-1]
        image = Image.open(path)
        image = image.resize((224,224))
        image = np.asarray(image)
        #x = image.reshape((1,224,224,3))
        row_num = i*4 + num_image -7
        x_test[row_num,:,:,:] = image
        y_test[row_num] = people
    i += 1
    
x_train.shape, y_train.shape, x_test.shape, y_test.shape

index_train = np.random.permutation(2688)
index_test = np.random.permutation(896)
x_train, y_train = x_train[index_train], y_train[index_train]
x_test, y_test = x_test[index_test], y_test[index_test]

x_train, x_test = tf.cast(x_train, tf.float32)/255.0, tf.cast(x_test, tf.float32)/255.0
y_train, y_test = tf.one_hot(y_train-1, depth=224), tf.one_hot(y_test-1, depth=224)

In [7]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras import Model

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten

from tensorflow.keras import Model
pre_trained_model = VGG16(input_shape=(224,224,3), weights='imagenet')
vgg_model = Model(inputs=pre_trained_model.input,outputs=pre_trained_model.layers[-2].output)


# First time, train without adjusting the ResNet weights to train the final dense layer first
vgg_model.trainable =  False
model = tf.keras.Sequential([
                             vgg_model,
                             Flatten(),
                             tf.keras.layers.Dense(512, activation='relu'),
                             tf.keras.layers.Dense(224, activation='softmax')
                                      ])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_2 (Functional)         (None, 4096)              134260544 
_________________________________________________________________
flatten_2 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               2097664   
_________________________________________________________________
dense_5 (Dense)              (None, 224)               114912    
Total params: 136,473,120
Trainable params: 2,212,576
Non-trainable params: 134,260,544
_________________________________________________________________


In [8]:
vgg_model.trainable =  True
model.add_loss(lambda: 0.01*tf.reduce_sum(tf.square(model.layers[-1].kernel)))

model.compile(optimizer=tf.keras.optimizers.Adam(0.0002),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=tf.keras.metrics.CategoricalAccuracy())
model.fit(x_train, y_train, batch_size=24, epochs=1, validation_split=0.2)

90/90 [==============================] - 78s 812ms/step - loss: 8.4863 - categorical_accuracy: 0.0040 - val_loss: 7.6458 - val_categorical_accuracy: 0.0019


In [10]:
# Manually stop at 20th epoch
model.fit(x_train, y_train, batch_size=24, epochs=25,validation_split=0.2)

Epoch 1/25
90/90 [==============================] - 71s 785ms/step - loss: 7.0135 - categorical_accuracy: 0.0060 - val_loss: 6.7680 - val_categorical_accuracy: 0.0000e+00
Epoch 2/25
90/90 [==============================] - 70s 782ms/step - loss: 6.5577 - categorical_accuracy: 0.0037 - val_loss: 6.3930 - val_categorical_accuracy: 0.0000e+00
Epoch 3/25
90/90 [==============================] - 70s 780ms/step - loss: 6.1972 - categorical_accuracy: 0.0070 - val_loss: 6.1209 - val_categorical_accuracy: 0.0019
Epoch 4/25
90/90 [==============================] - 70s 778ms/step - loss: 5.8114 - categorical_accuracy: 0.0153 - val_loss: 5.7824 - val_categorical_accuracy: 0.0130
Epoch 5/25
90/90 [==============================] - 70s 778ms/step - loss: 5.2569 - categorical_accuracy: 0.0609 - val_loss: 5.1917 - val_categorical_accuracy: 0.0706
Epoch 6/25
90/90 [==============================] - 70s 776ms/step - loss: 4.4661 - categorical_accuracy: 0.1363 - val_loss: 4.5167 - val_categorical_accurac

KeyboardInterrupt: 

In [11]:
model.evaluate(x_test, y_test)

28/28 [==============================] - 17s 241ms/step - loss: 1.9547 - categorical_accuracy: 0.7132


[1.9546836614608765, 0.7131696343421936]